In [1]:
from pathlib import Path
import sys
# Project path setup
project_root = Path.cwd()
if (project_root / 'src').exists():
    pass
else:
    # Fallback: user-specific path
    project_root = Path('/home/chu034/Yaohang_Li/cDiff')


if not project_root.exists() or not (project_root / 'src').exists():
    raise FileNotFoundError(f"找不到项目目录或 src: {project_root}")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"项目路径: {project_root}")
print("Python 路径已更新")


项目路径: /home/chu034/Yaohang_Li/cDiff
Python 路径已更新


In [6]:
# Config: set your trained checkpoint info here
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from datasets import load_dataset
from models.neural_sampler import DiffusionPosteriorSampler, NormalizingFlowPosteriorSampler
from utils import load_torch_model, SET_SEED

# ---- User-editable ----
DATASET = "circle"            # circle dataset
MODEL = "Diffusion"           # "Diffusion" or "NormalizingFlow"
USE_ENCODER = True            # circle uses set of (x,y) -> use encoder
DATA_TYPE = "iid"             # encoder type used in training (iid/time/set)

SAVE_ROOT = "/home/chu034/Yaohang_Li/cDiff/result"        # root path used during training (see main_circle.py comment)
EPOCHS_NAME = "5000"          # model_name token, usually epochs (e.g., "5000")
SEED = 1                      # seed index used during training
N_POST_SAMPLES = 1000         # how many posterior samples to draw
DIFFUSION_STEPS = 18          # steps for Diffusion sampler
DEVICE_INDEX = 0              # GPU index; falls back to CPU if no CUDA
N_OBS_POINTS = None           # if None, dataset picks random size per batch
BATCH_SIZE = 2                # small batch just to fetch one observation
# -----------------------

# Compute resolved paths/types
DEVICE = torch.device(f"cuda:{DEVICE_INDEX}" if torch.cuda.is_available() else "cpu")
MODEL_TYPE = MODEL
SAVE_DIR = os.path.join(SAVE_ROOT, DATASET)

print(f"Device: {DEVICE}")
print(f"Loading from: {SAVE_DIR}")


Device: cuda:0
Loading from: /home/chu034/Yaohang_Li/cDiff/result/circle


In [7]:
# Build the model by inferring dimensions from a tiny dataloader
# Then load the trained weights
SET_SEED(42)

dataset_generator, _, _ = load_dataset(DATASET)

# Create a tiny loader to infer y/theta shapes; for encoder we don't set n_sample
if USE_ENCODER:
    dl = dataset_generator(n_batches=1, batch_size=BATCH_SIZE, return_ds=False)
else:
    # If no encoder, dataset expects n_sample=1 to get shape [B, 1, y_dim] collapsed later
    dl = dataset_generator(n_batches=1, batch_size=BATCH_SIZE, n_sample=1, return_ds=False)

theta_batch, y_batch = next(iter(dl))

y_dim = y_batch.shape[-1]
theta_dim = theta_batch.shape[1]

print(f"theta_dim: {theta_dim}, y_dim: {y_dim}")

# Construct model skeleton
if MODEL == "Diffusion":
    # circle is simple; use fixed sigma_data=0.5 as in EDM unless you trained differently
    model = DiffusionPosteriorSampler(
        y_dim=y_dim,
        x_dim=theta_dim,
        n_summaries=256,
        num_hidden_layer=4,
        device=DEVICE,
        use_encoder=USE_ENCODER,
        data_type=DATA_TYPE,
        sigma_data=0.5,
    )
elif MODEL == "NormalizingFlow":
    model = NormalizingFlowPosteriorSampler(
        y_dim=y_dim,
        x_dim=theta_dim,
        n_summaries=256,
        hidden_dim_decoder=32,
        n_flows_decoder=32,
        alpha=0.1,
        device=DEVICE,
        use_encoder=USE_ENCODER,
        data_type=DATA_TYPE,
    ).to(DEVICE)
else:
    raise NotImplementedError("MODEL should be 'Diffusion' or 'NormalizingFlow'")

# Load weights with naming convention used in utils.py
# File: {SAVE_DIR}/{EPOCHS_NAME}_seed={SEED}_{MODEL_TYPE}.pth
model = load_torch_model(model, SAVE_DIR, EPOCHS_NAME, SEED, MODEL_TYPE)
model = model.to(DEVICE)
model.eval()


theta_dim: 1, y_dim: 2
Encoder is for iid data. If not, please check it.
Model loaded from /home/chu034/Yaohang_Li/cDiff/result/circle/5000_seed=1_Diffusion.pth


DiffusionPosteriorSampler(
  (summary): DeepSetSummary(
    (instance_embedder): InstanceEmbedder(
      (embedder): Sequential(
        (0): Linear(in_features=2, out_features=256, bias=True)
        (1): SiLU()
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): SiLU()
        (4): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (rho): Sequential(
      (0): Linear(in_features=261, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=256, bias=True)
    )
  )
  (decoder): ScoreNetwork(
    (embed): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=16, out_features=32, bias=True)
      (2): Mish()
      (3): Linear(in_features=32, out_features=16, bias=True)
    )
    (layers): MLPNetwork(
      (layers): ModuleList(
        (0): Linear(in_features=273, out_features=256, bias=True)
        (1

In [9]:
model.sample(2, num_steps=DIFFUSION_STEPS)

AttributeError: 'int' object has no attribute 'shape'

In [10]:
y_batch

tensor([[[-0.2147, -1.8893],
         [-1.5477, -1.1046],
         [ 1.0587,  1.5794],
         ...,
         [ 0.2263, -1.8879],
         [ 1.7881, -0.6467],
         [ 1.1142, -1.5408]],

        [[-0.4710,  0.1513],
         [ 0.3405,  0.3588],
         [ 0.4742, -0.1408],
         ...,
         [-0.1419,  0.4739],
         [ 0.0509,  0.4921],
         [ 0.4775,  0.1293]]], device='cuda:0')

In [8]:
# Prepare one observed y and draw posterior samples for z
import torch

@torch.no_grad()
def prepare_observation(y_batch, use_encoder: bool, L: int):
    """
    Prepare conditioning tensor for model.sample.
    - With encoder: expect y as [B, n_points, y_dim]; take first and repeat to [L, n_points, y_dim].
    - Without encoder: expect y as [B, y_dim]; take first and repeat to [L, y_dim].
    """
    y0 = y_batch[0]
    if y0.ndim == 1:
        y0 = y0.unsqueeze(0)  # [y_dim] -> [1, y_dim]
    return y0.repeat(L, *([1] * (y0.ndim - 1)))

# Move data to device
y_batch = y_batch.to(DEVICE)

# Build conditioning for a single observed dataset
y_cond = prepare_observation(y_batch, use_encoder=USE_ENCODER, L=N_POST_SAMPLES)

# Sample posterior draws
if MODEL == "Diffusion":
    theta_samples = model.sample(y_cond, num_steps=DIFFUSION_STEPS)
else:
    theta_samples = model.sample(y_cond)

theta_samples = theta_samples.detach().cpu().numpy()  # shape [L, 1] for circle

print(f"theta_samples shape: {theta_samples.shape}")
print(f"mean={theta_samples.mean():.4f}, std={theta_samples.std():.4f}")


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# Plot: posterior histogram of z and the observed (x, y) points
import matplotlib.pyplot as plt

# Observed points (take the first set)
y_obs = y_batch[0].detach().cpu().numpy()  # shape [n_points, 2] for circle

# A crude reference radius from observed points (mean radius)
r_est = np.sqrt((y_obs[:, 0] ** 2 + y_obs[:, 1] ** 2)).mean()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: histogram of posterior z
axes[0].hist(theta_samples.squeeze(), bins=40, alpha=0.8, color='steelblue', edgecolor='k')
axes[0].axvline(r_est, color='red', linestyle='--', label=f"r_est≈{r_est:.3f}")
axes[0].set_title("Posterior samples of z (radius)")
axes[0].set_xlabel("z")
axes[0].set_ylabel("Frequency")
axes[0].legend()

# Right: scatter of observed (x, y)
axes[1].scatter(y_obs[:, 0], y_obs[:, 1], s=8, alpha=0.7)
axes[1].set_aspect('equal', 'box')
axes[1].set_title("Observed (x, y) points")
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")

plt.tight_layout()
plt.show()


In [ ]:
# Optional: save samples and the figure to disk
import pandas as pd

OUT_DIR = "./samples_circle"
os.makedirs(OUT_DIR, exist_ok=True)

base = f"{DATASET}_{MODEL_TYPE}_seed{SEED}_L{N_POST_SAMPLES}"
np.save(os.path.join(OUT_DIR, base + ".npy"), theta_samples)
pd.DataFrame(theta_samples, columns=["z"]).to_csv(os.path.join(OUT_DIR, base + ".csv"), index=False)

print("Saved:")
print(os.path.join(OUT_DIR, base + ".npy"))
print(os.path.join(OUT_DIR, base + ".csv"))
